In [ ]:
import time
import numpy as np
import pandas as pd
from sklearn import metrics
from tabulate import tabulate
from datetime import timedelta
import matplotlib.pyplot as plt

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import UnivariateFeatureSelector
from pyspark.ml.feature import ChiSqSelector, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier as pyspark_DecisionTreeClassifier

In [ ]:
# Creating a new spark session
spark = SparkSession.builder.master("local[*]").appName("MLlib lab").getOrCreate()

In [ ]:
# load in preprocessed data
sparkDF = spark.read.option("maxColumns", 22285).parquet("/mnt/the-data-transformers/spark_output/preprocessed_data_final2.pqt")

In [ ]:
'''
Function: spark_UnivariateFeatureSelector
INPUT:
------
i: Number of Features to select
_vector_sparkDF: DataFrame from which the Features will be selected

OUTPUT:
-------
1. The model's list of feature importances
'''
def spark_UnivariateFeatureSelector(i, _vector_sparkDF):
  # Selecting the best features from the entire dataset/chunk passed in
  selector = UnivariateFeatureSelector(
    featuresCol="features", 
    outputCol="selectedFeatures", 
    labelCol="type", 
    selectionMode="numTopFeatures"
  )
  selector.setFeatureType("continuous").setLabelType("categorical").setSelectionThreshold(i)
  result = selector.fit(_vector_sparkDF).transform(_vector_sparkDF)

  # Using pyspark DecisionTreeClassifier to define and fit the ML model
  dt = pyspark_DecisionTreeClassifier(labelCol="type", featuresCol="selectedFeatures")
  model = dt.fit(result)
  
  return model.featureImportances 


In [ ]:
'''
Function: obtain_accuracy
INPUT:
------
sparkDF: the spark DataFrame consisting of the dataset

OUTPUT:
-------
1. The accuracy returned by the evaluator
'''
def obtain_accuracy(sparkDF):
  # Make predictions
  selector = UnivariateFeatureSelector(
    featuresCol="features", 
    outputCol="selectedFeatures", 
    labelCol="type", 
    selectionMode="numTopFeatures"
  )

  vecAssembler = VectorAssembler(inputCols=sparkDF.columns, outputCol="features")
  sparkDF = vecAssembler.transform(sparkDF)
  
  selector.setFeatureType("continuous").setLabelType("categorical") #.setSelectionThreshold(i)
  result = selector.fit(sparkDF).transform(sparkDF)

  # Splitting the data into training & testing
  (train, test) = result.randomSplit([0.8, 0.2])

  # Using pyspark DecisionTreeClassifier to define and fit the ML model
  dt = pyspark_DecisionTreeClassifier(labelCol="type", featuresCol="selectedFeatures")
  model = dt.fit(train)
  predictions = model.transform(test)

  # Evaluating the predictions
  evaluator = MulticlassClassificationEvaluator(
    labelCol="type", 
    predictionCol="prediction", 
    metricName="accuracy"
  )

  accuracy = evaluator.evaluate(predictions)
  return accuracy

In [ ]:
'''
Function: get_important_features
INPUT:
------
num: the number of features present in a chunk/subset
i: the chunk number

OUTPUT:
-------
1. The features selected by the UnivariateFeatureSelector
'''
def get_important_features(num, i):
  num_columns = num
  num_features = num
  
  # get chunk using indexing
  print("range, lower: "+str(1+(i*num_columns))+ " upper:"+str(num_columns*(i+1)))
  chunk =  sparkDF.select(*sparkDF.columns[1+(i*num_columns):num_columns*(i+1)], sparkDF['type']).distinct()
  vecAssembler = VectorAssembler(inputCols=chunk.columns, outputCol="features")
  newDF = vecAssembler.transform(chunk)
  selected_features = spark_UnivariateFeatureSelector(num_features, newDF)

  return selected_features

'''
Function: remove_features
INPUT:
------
sparkDF: the spark DataFrame consisting of the dataset

OUTPUT:
-------
1. The new spark DataFrame that only contains the most important subset of features
'''
def remove_features(sparkDF, columns_input, chunks_input):
  print("removing features")
  columns = columns_input # 2000
  chunks = chunks_input # 11
  toDrop = []
  
  # loop through chunks
  for i in range(0, chunks):
    print("chunk: "+str(i+1))

    # set num columns to amount left if below set columns
    if (len(sparkDF.columns) < columns):
      print("updating remaining columns")
      columns = len(sparkDF.columns)
      
    # get important features
    important_feature_vectors = get_important_features(columns, i) 
    vector = important_feature_vectors.toArray()
    index  = columns*i
    
    # loop through all colums and and add low importance ones to array
    for c in range(0, columns):
      if (vector[c] == 0): # if unimportant
        tmp_index = index + c
        if (sparkDF.columns[tmp_index] and sparkDF.columns[tmp_index] != "type"): # if index is correct
          toDrop.append(sparkDF.columns[tmp_index])
        
  # drop low importance columns
  sparkDF = sparkDF.drop(*toDrop)
  print("remaining columns: "+str(len(sparkDF.columns)))

  return sparkDF

Local Approach

In [ ]:
# local approach - running univariate selector on every chunk of the dataset whilst aggregating the results
sparkDF = remove_features(sparkDF, 2000, 11)

removing features
chunk: 1
range, lower: 1 upper:2000
chunk: 2
range, lower: 2001 upper:4000
chunk: 3
range, lower: 4001 upper:6000
chunk: 4
range, lower: 6001 upper:8000
chunk: 5
range, lower: 8001 upper:10000
chunk: 6
range, lower: 10001 upper:12000
chunk: 7
range, lower: 12001 upper:14000
chunk: 8
range, lower: 14001 upper:16000
chunk: 9
range, lower: 16001 upper:18000
chunk: 10
range, lower: 18001 upper:20000
chunk: 11
range, lower: 20001 upper:22000
remaining columns: 327

In [ ]:
# count number of features selected
len(sparkDF.columns)

Out[10]: 327

In [ ]:
# check accuracy of the local solution's model
local_solution_accuracy = obtain_accuracy(sparkDF)
local_solution_accuracy

In [ ]:
local_solution_accuracy

Out[19]: 1.0

Global Approach

In [ ]:
# reset sparkDF to contain the initial preprocessed data again to run for the global solution
sparkDF = spark.read.option("maxColumns", 22285).parquet("/mnt/the-data-transformers/spark_output/preprocessed_data_final2.pqt")

In [ ]:
# global approach - running univariate selector on 1 chunk containing the entire dataset (no aggregation required)
sparkDF = remove_features(sparkDF, 22000, 1)

removing features
chunk: 1
range, lower: 1 upper:22000

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3841274923039632> in <module> 
 1 # global approach - running univariate selector on 1 chunk containing the entire dataset 
 ----> 2 sparkDF = remove_features ( sparkDF , 22000 , 1 ) 

 <command-3841274923039619> in remove_features (sparkDF, columns_input, chunks_input) 
 30 
 31 # get important features 
 ---> 32 important_feature_vectors = get_important_features ( columns , i ) 
 33 vector = important_feature_vectors . toArray ( ) 
 34 index = columns * i

 <command-3841274923039619> in get_important_features (num, i) 
 9 vecAssembler = VectorAssembler ( inputCols = chunk . columns , outputCol = "features" ) 
 10 newDF = vecAssembler . transform ( chunk ) 
 ---> 11 selected_features = spark_UnivariateFeatureSelector ( num_features , newDF ) 
 12 
 13 return selected_features

 <command-3841274923039618> in spark_UnivariateFeatureSelector (i, _vector_sparkDF) 
 26 # Using pyspark DecisionTreeClassifier to define and fit the ML model 
 27 dt = pyspark_DecisionTreeClassifier ( labelCol = "type" , featuresCol = "selectedFeatures" ) 
 ---> 28 model = dt . fit ( result ) 
 29 
 30 return model . featureImportances

 /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py in patched_method (self, *args, **kwargs) 
 28 call_succeeded = False 
 29 try : 
 ---> 30 result = original_method ( self , * args , ** kwargs ) 
 31 call_succeeded = True 
 32 return result

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 159 return self . copy ( params ) . _fit ( dataset ) 
 160 else : 
 --> 161 return self . _fit ( dataset ) 
 162 else : 
 163 raise TypeError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 333 
 334 def _fit ( self , dataset ) : 
 --> 335 java_model = self . _fit_java ( dataset ) 
 336 model = self . _create_model ( java_model ) 
 337 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 330 """
 331 self . _transfer_params_to_java ( ) 
 --> 332 return self . _java_obj . fit ( dataset . _jdf ) 
 333 
 334 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o72538.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 12 in stage 1025.0 failed 4 times, most recent failure: Lost task 12.3 in stage 1025.0 (TID 3706) (10.139.64.7 executor 18): ExecutorLostFailure (executor 18 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2973)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2920)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGSc

In [ ]:
# count number of features selected
len(sparkDF.columns)

Out[18]: 327

In [ ]:
# check accuracy of the global solution's model
global_solution_accuracy = obtain_accuracy(sparkDF)
global_solution_accuracy